<a href="https://colab.research.google.com/github/Anugyagupta43/GenCloset/blob/main/GenCloset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("Womens Clothing E-Commerce Reviews.csv")  # Adjust path if needed
df.head()


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
# Drop unnamed index column if present
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Check nulls
df.isna().sum()


,0
Clothing ID,0
Age,0
Title,3810
Review Text,845
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,14
Department Name,14
Class Name,14


In [ ]:


df = df.dropna(subset=['Review Text']).reset_index(drop=True)


In [ ]:
df['Title'] = df['Title'].fillna("No Title")


In [ ]:
df = df.dropna(subset=['Division Name', 'Department Name', 'Class Name']).reset_index(drop=True)


In [ ]:
df.to_csv("cleaned_fashion_reviews.csv", index=False)
print("Saved as cleaned_fashion_reviews.csv")


Saved as cleaned_fashion_reviews.csv


In [ ]:
def generate_description(review_text):
    prompt = f"""
    Rewrite this customer review as a stylish product description for a fashion catalog:

    Review: "{review_text}"

    Product Description:
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=100
    )
    return response['choices'][0]['message']['content'].strip()


In [ ]:
pip install openai==0.28


In [ ]:
sample_review = df['Review Text'].iloc[0]
print("Original Review:", sample_review)
print("\nGenerated Description:", generate_description(sample_review))


Original Review: Absolutely wonderful - silky and sexy and comfortable


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
!pip install transformers torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from transformers import pipeline

# Load a text-to-text model for rewriting tasks
gen_model = pipeline("text2text-generation", model="google/flan-t5-small")

def local_generate_description(review_text):
    prompt = f"Turn this customer review into a stylish product description: {review_text}"
    result = gen_model(prompt, max_length=100, do_sample=True, temperature=0.7)
    return result[0]['generated_text']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
sample_review = df['Review Text'].iloc[0]
print("Original Review:", sample_review)
print("\nGenerated Description:", local_generate_description(sample_review))


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Original Review: Absolutely wonderful - silky and sexy and comfortable

Generated Description: Five Stars


In [ ]:
def rewrite_as_description(review_text):
    prompt = f"Rewrite this customer review as a stylish fashion product description:\n{review_text}"
    result = gen_model(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)
    return result[0]['generated_text']


In [ ]:
def playful_version(review_text):
    prompt = f"Rewrite this review in a fun and playful tone:\n{review_text}"
    result = gen_model(prompt, max_new_tokens=100, do_sample=True, temperature=0.9)
    return result[0]['generated_text']


In [ ]:
def formal_version(review_text):
    prompt = f"Rewrite this review using a formal and professional tone:\n{review_text}"
    result = gen_model(prompt, max_new_tokens=100, do_sample=False)
    return result[0]['generated_text']


In [ ]:
def suggest_styling_tip(review_text, category="Dresses"):
    prompt = f"""You're a fashion stylist. A customer reviewed a {category}.

Review: "{review_text}"

Based on this, give one stylish tip on how to wear this item:"""
    result = gen_model(prompt, max_new_tokens=60, do_sample=True, temperature=0.8)
    return result[0]['generated_text']


In [ ]:
def suggest_occasion(review_text, category="blouse"):
    prompt = f"""You're a fashion expert. Based on the following review of a {category}, suggest one specific occasion where it would be perfect to wear. Be short and direct.

Review: "{review_text}"

Occasion:"""
    result = gen_model(prompt, max_new_tokens=30, do_sample=True, temperature=0.8)
    return result[0]['generated_text']


In [ ]:
gen_model = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
sample_review = df['Review Text'].iloc[0]
print("👗 Description:\n", rewrite_as_description(sample_review))
print("\n💬 Playful:\n", playful_version(sample_review))
print("\n👠 Styling Tip:\n", suggest_styling_tip(sample_review))
print("\n🎉 Occasion:\n", suggest_occasion(sample_review))


👗 Description:
 Absolutely wonderful - silky and sexy and comfortable

💬 Playful:
 I'm not a big fan of sexy clothes, but this one is great. It's silky, sexy, and comfortable. It's a little on the sexy side, but it's not too sexy. It's not as sexy as other sexy sexy sexy sexy sexy sexy sexy 

📚 Formal:
 Absolutely wonderful - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and comfortable - silky and sexy and

👠 Styling Tip:
 Wear it with a tuxedo.

🎉 Occasion:
 Wearing it for a date.
